In [141]:
import sqlalchemy as db
from sqlalchemy import MetaData,Table,Column,inspect,select,join
import xml.etree.ElementTree as ET
import pandas as pd

In [142]:
engine = db.create_engine('sqlite:///Tracks.db')
conn= engine.connect()
metadata = db.MetaData()

# ARTIST
Artist = Table('Artist',metadata,
Column('id', db.Integer(),nullable=False,primary_key=True,autoincrement=True,unique=True),
Column('Artist_name',db.String(),nullable=False,unique=True)
)
Artist.drop(engine,checkfirst=True)  
Artist.create(engine,checkfirst=True) # it will check whether the table exists or not, then create 
# if not present 
## similarly  Artist.drop(engine,checkfirst=True)
### check if exists then drop table 

# ALBUM
Album = Table('Album',metadata,
Column('id', db.Integer(),nullable=False,primary_key=True,autoincrement=True,unique=True),
Column('Artist_id',db.Integer()),
Column('Album_name',db.String(),unique=True))

Album.drop(engine,checkfirst=True)
Album.create(engine,checkfirst=True)

# GENRE
Genre = Table('Genre',metadata,
Column('id', db.Integer(),nullable=False,primary_key=True,autoincrement=True,unique=True),
Column('Genre_name',db.String(),unique=True)
)
Genre.drop(engine,checkfirst=True)
Genre.create(engine,checkfirst=True)

# TRACK
Track = Table('Track',metadata,
Column('id', db.Integer(),nullable=False,primary_key=True,autoincrement=True,unique=True),
Column('Track_name',db.String(),unique=True),
Column('Length',db.Integer()),
Column('Play_count',db.Integer()),
Column('Rating',db.Integer()),
Column('Album_id',db.Integer(),db.ForeignKey('Album.id')),
Column('Genre_id',db.Integer(),db.ForeignKey('Genre.id'))
)
Track.drop(engine,checkfirst=True)
Track.create(engine,checkfirst=True)

# now no need to run metadata.create_all()  method


In [143]:
fname = 'Library.xml'
fh = open(fname)

data = ET.parse(fh)
dict_ele= data.findall('dict/dict/dict')

def extract(key_ele,key):
    found = False
    for child in key_ele:
        if found : return child.text
        if child.tag=='key' and child.text == key :
            found = True
    return None
    

for ele in dict_ele:
    
    title_name = extract(ele,'Name')
    genre_name = extract(ele,'Genre')
    album_name = extract(ele,'Album')
    rating = extract(ele,'Rating')
    count = extract(ele,'Play Count')
    artist = extract(ele,'Artist')
    length = extract(ele,'Size')
    
    if title_name is None or genre_name is None or album_name is None or rating is None or artist is None: continue
    
    try:
        query = db.insert(Artist).values(Artist_name = artist)
        res = conn.execute(query)
        query = db.select(Artist.columns.id).where(Artist.columns.Artist_name == artist)
        res = conn.execute(query).all()
        artist_id = res[0][0]
    except:
        None
    
    
    try:
        query = db.insert(Genre).values(Genre_name = genre_name)
        res = conn.execute(query)
        query = db.select(Genre.columns.id).where(Genre.columns.Genre_name == genre_name)
        res = conn.execute(query).all()
        genre_id = res[0][0]
    except:
        None
    
    
    try:
        query = db.insert(Album).values(Album_name = album_name,Artist_id = artist_id)
        res = conn.execute(query)
        query = db.select(Album.columns.id).where(Album.columns.Album_name == album_name)
        res = conn.execute(query).all()
        album_id = res[0][0]
        
    except:
        None
        
    try:
        query = db.insert(Track).values(Track_name = title_name,Length= length,Play_count= count,Rating= rating,Album_id=album_id,Genre_id=genre_id)
        res = conn.execute(query)
    except:
        None   
        

In [144]:
##  join operation using sqlalchemy
## https://www.tutorialspoint.com/sqlalchemy/sqlalchemy_core_using_joins.htm

result = conn.execute(db.select(Track)).fetchall()
df = pd.DataFrame(result)
df.columns = result[0].keys()
df.head(20)




,id,Track_name,Length,Play_count,Rating,Album_id,Genre_id
0,1,Another One Bites The Dust,4344295,55,100,1,1
1,2,Asche Zu Asche,4638526,79,100,2,2
2,3,Beauty School Dropout,4801377,48,100,3,3
3,4,Black Dog,5934629,109,100,4,3
4,5,Bring The Boys Back Home,1744588,33,100,5,3
5,6,Circles,7109552,54,60,6,4
6,7,Comfortably Numb,7684826,36,100,6,4
7,8,Crazy Little Thing Called Love,3274850,38,100,6,4
8,9,Electric Funeral,5862554,44,100,7,5
9,10,Fat Bottomed Girls,5152514,38,100,7,5


In [145]:
music_database = Table('MusicData',metadata,
Column('Title',db.String(128),unique=True),
Column('Length',db.Integer()),
Column('Artist',db.String(128),db.ForeignKey('Artist.id')),
Column('Genre',db.String(128),db.ForeignKey('Genre.id')),
Column('Album',db.String(128),db.ForeignKey('Album.id'))
)
music_database.drop(engine,checkfirst=True)

